# Compare two websites by taking a screenshot (shot-scraper) using ImageMagick

In [6]:
# where magick   # returns the location of imagemagick

# Note to myself: It looks like one has to work with brew when dealing with IM. Downloading it from source does not work on Mac with M1
# See:  https://stackoverflow.com/questions/9586048/imagemagick-no-decode-delegate
# When you download it with brew, all dependencies (JPEG, PNG) compatible for mac are automatically installed. At least thats what I think. 

# brew help
# brew list  # displays all the packaged downloaded with brew.


# ------------------ Practical Information

# If you installed imagemagick from source and want to uninstall it, do the following steps:
# cd ImageMagick-6.8.2-10 # your version
# ./configure
# make uninstall

In [ ]:
# pip install requests
# pip install subprocess
# pip install beautifulsoup4

# pip install shot-scraper
# /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
# brew install imagemagick

In [1]:
%ls

microspot/            project_niklas.ipynb  wikipedia/


In [4]:
# Version 1
import requests
from bs4 import BeautifulSoup
import subprocess
import os

# Version 2
import cv2
import numpy as np

## Version 1

In [8]:

def scrape_website(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract and print all the URLs
        for link in soup.find_all('a'):
            print(link.get('href'))
    except Exception as e:
        print(f"Failed to scrape {url} due to {e}")

def take_screenshot(url, filename):
    try:
        # Using subprocess to call shot-scraper command
        subprocess.run(['shot-scraper', url, '--output', filename], check=True)
    except Exception as e:
        print(f"Failed to take screenshot of {url} due to {e}")

def compare_screenshots(image1, image2, diff_image):
    try:
        # Using ImageMagick's compare tool
        subprocess.run(['compare', image1, image2, diff_image], check=True)
    except Exception as e:
        print(f"Failed to compare images due to {e}")

# URLs of the websites to scrape and take screenshots
url1 = 'https://en.wikipedia.org/wiki/Lion'
url2 = 'https://en.wikipedia.org/wiki/Tiger'

# url1 = 'https://www.microspot.ch/de/computer-gaming/notebooks/notebook--c511000/apple-macbook-pro-2023-14-2-apple-m3-pro-11-core-chip-18-gb-ram-512-gb-ssd--p0014039927'
# url2 = 'https://www.microspot.ch/de/computer-gaming/notebooks/notebook--c511000/apple-macbook-air-2024-13-6-apple-m3-chip-8-gb-ram-256-gb-ssd--p0014109929'


# Take screenshots
screenshot1 = 'screenshot1.png'
screenshot2 = 'screenshot2.png'
take_screenshot(url1, screenshot1)
take_screenshot(url2, screenshot2)

# Compare the screenshots
diff_image = 'difference.png'
compare_screenshots(screenshot1, screenshot2, diff_image)

print("Screenshots taken and comparison done. Check the difference image.")

Screenshot of 'https://www.microspot.ch/de/computer-gaming/notebooks/notebook--c511000/apple-macbook-pro-2023-14-2-apple-m3-pro-11-core-chip-18-gb-ram-512-gb-ssd--p0014039927' written to 'screenshot1.png'
Screenshot of 'https://www.microspot.ch/de/computer-gaming/notebooks/notebook--c511000/apple-macbook-air-2024-13-6-apple-m3-chip-8-gb-ram-256-gb-ssd--p0014109929' written to 'screenshot2.png'


Failed to compare images due to Command '['compare', 'screenshot1.png', 'screenshot2.png', 'difference.png']' returned non-zero exit status 1.
Screenshots taken and comparison done. Check the difference image.


## Version 2

In [14]:

def scrape_website(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract and print all the URLs
        for link in soup.find_all('a'):
            print(link.get('href'))
    except Exception as e:
        print(f"Failed to scrape {url} due to {e}")

def take_screenshot(url, filename):
    try:
        # Using subprocess to call shot-scraper command
        subprocess.run(['shot-scraper', url, '--output', filename], check=True)
    except Exception as e:
        print(f"Failed to take screenshot of {url} due to {e}")

def compare_screenshots(image1, image2, diff_image):
    try:
        # Using ImageMagick's compare tool
        subprocess.run(['compare', image1, image2, '-compose', 'src', diff_image], check=True)
        
        # Load the difference image
        diff_img = cv2.imread(diff_image)
        
        # Convert to grayscale
        gray_diff_img = cv2.cvtColor(diff_img, cv2.COLOR_BGR2GRAY)
        
        # Threshold to get binary image
        ret, thresh = cv2.threshold(gray_diff_img, 0, 255, cv2.THRESH_BINARY)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Draw unfilled rectangles around contours
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(diff_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Save the image with rectangles drawn
        cv2.imwrite(diff_image, diff_img)
        
    except Exception as e:
        print(f"Failed to compare images due to {e}")

# URLs of the websites to scrape and take screenshots
# url1 = 'https://en.wikipedia.org/wiki/Lion'
# url2 = 'https://en.wikipedia.org/wiki/Tiger'

url1 = 'https://www.microspot.ch/de/computer-gaming/notebooks/notebook--c511000/apple-macbook-pro-2023-14-2-apple-m3-pro-11-core-chip-18-gb-ram-512-gb-ssd--p0014039927'
url2 = 'https://www.microspot.ch/de/computer-gaming/notebooks/notebook--c511000/apple-macbook-air-2024-13-6-apple-m3-chip-8-gb-ram-256-gb-ssd--p0014109929'


# Take screenshots
screenshot1 = 'screenshot1.png'
screenshot2 = 'screenshot2.png'
take_screenshot(url1, screenshot1)
take_screenshot(url2, screenshot2)

# Compare the screenshots
diff_image = 'difference.png'
compare_screenshots(screenshot1, screenshot2, diff_image)

print("Screenshots taken and comparison done. Check the difference image.")

Screenshot of 'https://www.microspot.ch/de/computer-gaming/notebooks/notebook--c511000/apple-macbook-pro-2023-14-2-apple-m3-pro-11-core-chip-18-gb-ram-512-gb-ssd--p0014039927' written to 'screenshot1.png'
Screenshot of 'https://www.microspot.ch/de/computer-gaming/notebooks/notebook--c511000/apple-macbook-air-2024-13-6-apple-m3-chip-8-gb-ram-256-gb-ssd--p0014109929' written to 'screenshot2.png'


Failed to compare images due to Command '['compare', 'screenshot1.png', 'screenshot2.png', '-compose', 'src', 'difference.png']' returned non-zero exit status 1.
Screenshots taken and comparison done. Check the difference image.
